In [ ]:
#Simulations
import random
import pandas as pd

import tensorflow as tf #pip install "tensorflow<2.11"
from tensorflow.keras import layers
from collections import deque
import random
import numpy as np
import shap

import hrl_gym #simulate addiction
import gymnasium as gym

import shap

# Function Approximator (Neural Network)

In [ ]:
class Network(tf.keras.Model):  
    def __init__(self, input_size, nb_action):
        super().__init__()
        self.input_size = input_size
        self.nb_action = nb_action
        self.fc1 = tf.keras.layers.Dense(30, activation=tf.nn.relu, input_shape=(input_size,))
        self.fc2 = tf.keras.layers.Dense(nb_action)
    
    def call(self, state):
        x = self.fc1(state)
        x = tf.nn.relu(x)
        q_values = self.fc2(x)
        return q_values

# Experience Replay

In [ ]:
class ReplayMemory(): 
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
    
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
    
    def sample(self, batch_size):
        samples = random.sample(self.memory, batch_size)
        states, next_states, actions, rewards, dones = zip(*samples)
        states = tf.convert_to_tensor(states)
        next_states = tf.convert_to_tensor(next_states)
        actions = tf.convert_to_tensor(actions)
        rewards = tf.convert_to_tensor(rewards)
        dones = tf.convert_to_tensor(dones)
        return states, next_states, actions, rewards, dones

# Deep Q-Learning Agent

In [ ]:
class Dqn():
    def __init__(self, input_size, nb_action, gamma, capacity=100000, learning=0.001, temperature=100, sample_rate=100):
        self.gamma = gamma
        self.reward_window = []
        self.model = Network(input_size, nb_action)
        self.memory = ReplayMemory(capacity)
        self.sample_rate = sample_rate
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning)
        self.last_state = tf.zeros((1, input_size))
        self.last_action = 0
        self.last_reward = 0
        self.temp = temperature

        self.df = pd.DataFrame(columns=['outputs'])

    def select_action(self, state):
        q_value_tensor = self.model(state, training=False)
        q_values = q_value_tensor.numpy().squeeze()
        probs = tf.nn.softmax(q_value_tensor * self.temp)
        action_prob = probs.numpy().squeeze()

        action = tf.random.categorical(probs, num_samples=1)
        action = action.numpy()[0, 0]

        return action, q_values, action_prob

    def update(self, reward, new_signal):
        new_state = tf.convert_to_tensor([new_signal], dtype=tf.float32)
        self.memory.push((self.last_state, new_state, self.last_action, reward, False))

        self.reward_window.append(reward)
        if len(self.reward_window) > 1000:
            del self.reward_window[0]

        if len(self.memory.memory) > 128:
            states, next_states, actions

        # Compute SHAP values to explain the contribution of each input feature to the output of the neural network
        explainer = shap.KernelExplainer(self.model.predict_on_batch, shap.sample(states, 100))
        shap_values = explainer.shap_values(states, nsamples=10)

# Simulation